## quadratic_equation_workflow

In [2]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Literal


# defining state
class QuadState(TypedDict):
    a: int
    b: int
    c: int

    equation: str
    discriminant: float
    result: str



In [3]:
# defining nodes
def show_equation(state:QuadState):
    equation = f"{state['a']}x2{state['b']}x{state['c']}"
    return {"equation":equation}

def calculate_discriminant(state:QuadState):
    discriminant = state['b']**2 - (4*state['a']*state["c"]) # b2-4ac
    return {"discriminant":discriminant}

def real_roots(state: QuadState):

    root1 = (-state["b"] + state["discriminant"]**0.5)/(2*state["a"]) # -b + underroot of d / 2a
    root2 = (-state["b"] - state["discriminant"]**0.5)/(2*state["a"]) # # -b - underroot of d / 2a

    result = f'The roots are {root1} and {root2}'

    return {'result': result}

def repeated_roots(state: QuadState):

    root = (-state["b"])/(2*state["a"]) # -b/2a

    result = f'Only repeating root is {root}'

    return {'result': result}

def no_real_roots(state: QuadState):

    result = f'No real roots'

    return {'result': result}

def check_condition(state: QuadState) -> Literal["real_roots", "repeated_roots", "no_real_roots"]:
    if state['discriminant'] > 0:
        return "real_roots"
    elif state['discriminant'] == 0:
        return "repeated_roots"
    else:
        return "no_real_roots"
# 


In [4]:
# definig graph
graph = StateGraph(QuadState)

# adding nodes and edges
graph.add_node("show_equation", show_equation)
graph.add_node("calculate_discriminant", calculate_discriminant)
graph.add_node("real_roots", real_roots)
graph.add_node("repeated_roots", repeated_roots)
graph.add_node("no_real_roots", no_real_roots)

graph.add_edge(START,"show_equation")
graph.add_edge("show_equation","calculate_discriminant")

graph.add_conditional_edges("calculate_discriminant", check_condition )
graph.add_edge("real_roots", END)
graph.add_edge("repeated_roots", END)
graph.add_edge("no_real_roots", END)

# compile the graph
workflow = graph.compile()

# exicute the graph
state = {"a":4,"b":-5,"c":-4}
result = workflow.invoke(state)
print(result)

{'a': 4, 'b': -5, 'c': -4, 'equation': '4x2-5x-4', 'discriminant': 89, 'result': 'The roots are 1.8042476415070754 and -0.5542476415070754'}


## review_reply_workflow

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import START, END, StateGraph
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated, Literal
from dotenv import load_dotenv
import operator

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

# defing output schema
class SentementSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description='Sentiment of the review')

class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')


structured_model1 = model.with_structured_output(SentementSchema)
structured_model2 = model.with_structured_output(DiagnosisSchema)



In [6]:
prompt = 'What is the sentiment of the following review - The software too good'
structured_model1.invoke(prompt).sentiment

'positive'

In [7]:
# Define state 
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str

# defining nodes
def find_sentiment(state: ReviewState):

    prompt = f'For the following review find out the sentiment \n {state["review"]}'
    sentiment = structured_model1.invoke(prompt).sentiment

    return {'sentiment': sentiment}

def positive_response(state: ReviewState):
    prompt = f"""Write a warm thank-you message in response to this review:
    \n\n\"{state['review']}\"\n
Also, kindly ask the user to leave feedback on our website."""
    
    response = model.invoke(prompt).content

    return {'response': response}

def run_diagnosis(state: ReviewState):
    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""
    response = structured_model2.invoke(prompt)

    return {'diagnosis': response.model_dump()}

def negative_response(state: ReviewState):
    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
    response = model.invoke(prompt).content

    return {'response': response}

def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:

    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'



In [8]:
# defining the graph
graph = StateGraph(ReviewState)

# adding nodes and edges
graph.add_node("find_sentiment", find_sentiment)
graph.add_node("positive_response", positive_response)
graph.add_node("run_diagnosis", run_diagnosis)
graph.add_node("negative_response", negative_response)

graph.add_edge(START, "find_sentiment")
graph.add_conditional_edges("find_sentiment", check_sentiment)

graph.add_edge("run_diagnosis", "negative_response")
graph.add_edge("negative_response", END)
graph.add_edge("positive_response", END)

# compile the graph
workflow = graph.compile()

# exicue the graph
state = {'review': "I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality."}
result = workflow.invoke(state)
print(result)





{'review': 'I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.', 'sentiment': 'negative', 'diagnosis': {'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'}, 'response': "Subject: Re: Urgent - Bug Report - [Original Subject Line]\n\nHi [User Name],\n\nI'm so sorry to hear you're experiencing this issue and that it's causing you frustration. I understand how urgent this is, and I want to assure you I'll do everything I can to help resolve it quickly.\n\nCould you please provide me with a little more information?  The more details you can give me, the better I can understand the problem and find a solution.  Specifically, it would be helpful if you could tell me:\n\n* **What steps were you taking when the bug occurred?**  A step-by-step description would be ideal.\n* **What exactly happened?**  Please 